In [1]:
import sys
import random
import math
import os     #
import pyaudio
from scipy import signal
# import pygame
from socket import *
# from pygame.locals import *
from random import * 
import numpy as np
from scipy.signal import blackmanharris, fftconvolve
from numpy import argmax, sqrt, mean, diff, log
# from matplotlib.mlab import find

## Globals

In [2]:
def find(condition):
    res, = np.nonzero(np.ravel(condition))
    return res

def loudness(chunk):
    data = np.array(chunk, dtype=float) / 32768.0
    ms = math.sqrt(np.sum(data ** 2.0) / len(data))
    if ms < 10e-8: ms = 10e-8
    return 10.0 * math.log(ms, 10.0)

def parabolic(f, x): 
    xv = 1/2. * (f[x-1] - f[x+1]) / (f[x-1] - 2 * f[x] + f[x+1]) + x
    yv = f[x] - 1/4. * (f[x-1] - f[x+1]) * (xv - x)
    return (xv, yv)

def find_nearest(array, value):
    index = (np.abs(array - value)).argmin()
    return array[index]

def closest_value_index(array, guessValue):
    # Find closest element in the array, value wise
    closestValue = find_nearest(array, guessValue)
    # Find indices of closestValue
    indexArray = np.where(array==closestValue)
    # Numpys 'where' returns a 2D array with the element index as the value
    return indexArray[0][0]
    

def freq_from_autocorr(raw_data_signal, fs):                          
    corr = fftconvolve(raw_data_signal, raw_data_signal[::-1], mode='full')
    corr = corr[round(len(corr)/2):]
    d = diff(corr)
    start = find(d > 0)[0]
    peak = argmax(corr[start:]) + start
    px, py = parabolic(corr, peak)
    return fs / px 

def build_default_tuner_range():
    
    return {65.41:'C2', 
            69.30:'C2#',
            73.42:'D2',  
            77.78:'E2b', 
            82.41:'E2',  
            87.31:'F2',  
            92.50:'F2#',
            98.00:'G2', 
            103.80:'G2#',
            110.00:'A2', 
            116.50:'B2b',
            123.50:'B2', 
            130.80:'C3', 
            138.60:'C3#',
            146.80:'D3',  
            155.60:'E3b', 
            164.80:'E3',  
            174.60:'F3',  
            185.00:'F3#',
            196.00:'G3',
            207.70:'G3#',
            220.00:'A3',
            233.10:'B3b',
            246.90:'B3', 
            261.60:'C4', 
            277.20:'C4#',
            293.70:'D4', 
            311.10:'E4b', 
            329.60:'E4', 
            349.20:'F4', 
            370.00:'F4#',
            392.00:'G4',
            415.30:'G4#',
            440.00:'A4',
            466.20:'B4b',
            493.90:'B4', 
            523.30:'C5', 
            554.40:'C5#',
            587.30:'D5', 
            622.30:'E5b', 
            659.30:'E5', 
            698.50:'F5', 
            740.00:'F5#',
            784.00:'G5',
            830.60:'G5#',
            880.00:'A5',
            932.30:'B5b',
            987.80:'B5', 
            1047.00:'C6',
            1109.0:'C6#',
            1175.0:'D6', 
            1245.0:'E6b', 
            1319.0:'E6', 
            1397.0:'F6', 
            1480.0:'F6#',
            1568.0:'G6',
            1661.0:'G6#',
            1760.0:'A6',
            1865.0:'B6b',
            1976.0:'B6', 
            2093.0:'C7'
            } 

## Sound Recorder

In [3]:
# See http://www.swharden.com/blog/2013-05-09-realtime-fft-audio-visualization-with-python/
class SoundRecorder:
        
    def __init__(self):
        self.RATE=48000
        self.BUFFERSIZE=3072 #1024 is a good buffer size 3072 works for Pi
        self.secToRecord=0.05
        self.threadsDieNow=False
        self.newAudio=False
        
    def setup(self):
        self.buffersToRecord=int(self.RATE*self.secToRecord/self.BUFFERSIZE)
        if self.buffersToRecord==0: self.buffersToRecord=1
        self.samplesToRecord=int(self.BUFFERSIZE*self.buffersToRecord)
        self.chunksToRecord=int(self.samplesToRecord/self.BUFFERSIZE)
        self.secPerPoint=1.0/self.RATE
        self.p = pyaudio.PyAudio()
        info = self.p.get_host_api_info_by_index(0)
        numdevices = info.get('deviceCount')
        # for i in range(0, numdevices):
            # if (self.p.get_device_info_by_host_api_device_index(0, i).get('maxInputChannels')) > 0:
                # print("Input Device id ", i, " - ", self.p.get_device_info_by_host_api_device_index(0, i).get('name'))
        

        self.inStream = self.p.open(format=pyaudio.paInt16,channels=1,rate=self.RATE,input=True,input_device_index=1, frames_per_buffer=self.BUFFERSIZE)
        self.xsBuffer=np.arange(self.BUFFERSIZE)*self.secPerPoint
        self.xs=np.arange(self.chunksToRecord*self.BUFFERSIZE)*self.secPerPoint
        self.audio=np.empty((self.chunksToRecord*self.BUFFERSIZE),dtype=np.int16)               
    
    def close(self):
        self.p.close(self.inStream)
    
    def getAudio(self):
        audioString=self.inStream.read(self.BUFFERSIZE)
        self.newAudio=True
        return np.fromstring(audioString,dtype=np.int16)

In [4]:
import time
tunerNotes = build_default_tuner_range()
frequencies = np.array(sorted(tunerNotes.keys()))
soundgate = 19

In [5]:
SR=SoundRecorder()                          # recording device (usb mic)
iterations = 0

while True:
    SR.setup()

    raw_data_signal = SR.getAudio()                                         #### raw_data_signal is the input signal data 
    signal_level = round(abs(loudness(raw_data_signal)),2)                  #### find the volume from the audio sample
    inputnote = round(freq_from_autocorr(raw_data_signal,SR.RATE),2)    #### find the freq from the audio sample

    if inputnote > frequencies[len(tunerNotes)-1]:                        #### not interested in notes above the notes list
        continue
        
    if inputnote < frequencies[0]:                                     #### not interested in notes below the notes list
        continue    

    if signal_level > soundgate:
        continue
    
    print(raw_data_signal)
    targetnote = closest_value_index(frequencies, round(inputnote, 2))
    print("Frequency: {} ".format(inputnote), end=" ")
    print("Input Note: {} ".format(tunerNotes[frequencies[targetnote]]))
    SR.close()


C:\Users\harri\AppData\Local\Temp\ipykernel_20800\3057157392.py:36: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(audioString,dtype=np.int16)


[224 189 153 ... 585 553 483]
Frequency: 109.22  Input Note: A2 
[ 48  43  35 ... 924 927 932]
Frequency: 116.64  Input Note: B2b 
[-1189 -1211 -1232 ...  1693  1497  1310]
Frequency: 118.29  Input Note: B2b 
[-255 -508 -780 ...  -84  -68  -57]
Frequency: 118.17  Input Note: B2b 
[-158 -156 -169 ...  401  378  382]
Frequency: 99.55  Input Note: G2 
[-173 -201 -233 ...  -38  -84 -128]
Frequency: 350.2  Input Note: F4 
[ 672  677  679 ... -565 -565 -566]
Frequency: 385.76  Input Note: G4 
[   0    0   -1 ... 1201 1120 1060]
Frequency: 669.79  Input Note: E5 
[503 433 374 ...   2   4   4]
Frequency: 190.87  Input Note: G3 
[  84  159  248 ... -873 -898 -930]
Frequency: 135.52  Input Note: C3# 
[-26 -21 -16 ... 352 446 523]
Frequency: 142.82  Input Note: D3 
[-2713 -2983 -3249 ...  -384  -350  -241]
Frequency: 116.63  Input Note: B2b 
[ -946 -1006 -1025 ... -1044  -999  -918]
Frequency: 103.46  Input Note: G2# 
[ 785 1185 1528 ... -113 -112 -109]
Frequency: 706.31  Input Note: F5 
[-19 -30

KeyboardInterrupt: 